In [1]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model
import pandas as pd


from sklearn.ensemble import RandomForestRegressor

%matplotlib inline

In [2]:
# import data

features = pd.read_csv('data/X_train.csv', sep=',')
targets = pd.read_csv('data/y_train.csv', sep=',')

print (targets)
print (features.head)


      carID  price
0     13207  31995
1     17314   7700
2     12342  58990
3     13426  12999
4     16004  16990
...     ...    ...
4955  16898  17999
4956  14416  28900
4957  15453   8998
4958  14666  23198
4959  15135  14900

[4960 rows x 2 columns]
<bound method NDFrame.head of       carID     brand      model  year transmission  mileage fuelType    tax  \
0     13207    hyundi   Santa Fe  2019    Semi-Auto     4223   Diesel  145.0   
1     17314  vauxhall        GTC  2015       Manual    47870   Diesel  125.0   
2     12342      audi        RS4  2019    Automatic     5151   Petrol  145.0   
3     13426        vw   Scirocco  2016    Automatic    20423   Diesel   30.0   
4     16004     skoda      Scala  2020    Semi-Auto     3569   Petrol  145.0   
...     ...       ...        ...   ...          ...      ...      ...    ...   
4955  16898      merc   GL Class  2015    Automatic    24314   Diesel  125.0   
4956  14416       bmw   6 Series  2017    Automatic    18000   Diesel  145.0 

In [3]:

# one hot encoding of categorical data 
# transmission

print(features.columns.tolist())
one_hot = pd.get_dummies(features['transmission'])
print (one_hot)
print (features['transmission'])
features = features.drop('transmission', axis = 1)

features = features.join(one_hot)


# remove brand and model
features = features.drop('brand', axis = 1)
features = features.drop('model', axis = 1)
print (features.head)


['carID', 'brand', 'model', 'year', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize']
      Automatic  Manual  Other  Semi-Auto
0             0       0      0          1
1             0       1      0          0
2             1       0      0          0
3             1       0      0          0
4             0       0      0          1
...         ...     ...    ...        ...
4955          1       0      0          0
4956          1       0      0          0
4957          0       1      0          0
4958          0       0      0          1
4959          1       0      0          0

[4960 rows x 4 columns]
0       Semi-Auto
1          Manual
2       Automatic
3       Automatic
4       Semi-Auto
          ...    
4955    Automatic
4956    Automatic
4957       Manual
4958    Semi-Auto
4959    Automatic
Name: transmission, Length: 4960, dtype: object
<bound method NDFrame.head of       carID  year  mileage fuelType    tax   mpg  engineSize  Automatic  \
0     13207  2019

In [4]:
# one hot fuelType
# Both fueltype and transmission have an "other" category, complicating thigs
#features['Other'].value_counts()


one_hot = pd.get_dummies(features['fuelType'])


features = features.drop('fuelType', axis = 1)
features = features.join(one_hot, how='left', lsuffix='_tr', rsuffix='_fuel')

# remove carID
features = features.drop('carID', axis = 1)
targets = targets.drop('carID', axis = 1)

# Test without/ with tax feature
#features = features.drop('tax', axis = 1)

In [5]:
x = features.copy()
y = targets.copy()

print (features)

      year  mileage    tax   mpg  engineSize  Automatic  Manual  Other_tr  \
0     2019     4223  145.0  39.8         2.2          0       0         0   
1     2015    47870  125.0  60.1         2.0          0       1         0   
2     2019     5151  145.0  29.1         2.9          1       0         0   
3     2016    20423   30.0  57.6         2.0          1       0         0   
4     2020     3569  145.0  47.1         1.0          0       0         0   
...    ...      ...    ...   ...         ...        ...     ...       ...   
4955  2015    24314  125.0  56.6         2.1          1       0         0   
4956  2017    18000  145.0  51.4         3.0          1       0         0   
4957  2015    84932   30.0  60.1         2.0          0       1         0   
4958  2017    30150  145.0  62.8         3.0          0       0         0   
4959  2014    40230  150.0  56.5         3.0          1       0         0   

      Semi-Auto  Diesel  Electric  Hybrid  Other_fuel  Petrol  
0          

In [6]:
print(x.shape)
print (y.shape)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(x)
scaled_x = scaler.transform(x)

(4960, 14)
(4960, 1)


In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(scaled_x, y, test_size=0.1, random_state=1)

In [8]:
model = RandomForestRegressor(max_depth=8, random_state=0)
model.fit(x_train, y_train.values.ravel())


RandomForestRegressor(max_depth=8, random_state=0)

In [9]:
y_prediction =  model.predict(x_test)
y_prediction.shape
#percentage error
from sklearn.metrics import mean_absolute_percentage_error
print("mean absolute percentage error is ==",mean_absolute_percentage_error(y_test,y_prediction)*100, "%")

mean absolute percentage error is == 15.730051511641102 %


In [10]:
from sklearn.metrics import mean_absolute_error
print("mean absolute error is ==",mean_absolute_error(y_test,y_prediction))

mean absolute error is == 2882.520494416112


In [11]:
from sklearn.metrics import r2_score
print("r2 score is ==",  r2_score(y_test,y_prediction))

r2 score is == 0.9233742679418723
